In [5]:
!pip install pvlib pandas numpy

In [6]:
import math
import requests
import pandas as pd
import numpy as np
import pvlib
from pvlib import location, solarposition, irradiance, pvsystem, modelchain
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS
import traceback
import sys

# ---------------------------
# Parâmetros do projeto
# ---------------------------
potencia_modulo = 605  # W
num_modulos = 22

lat, lon = -22.841432, -51.957627 #Colorado
#lat, lon = -8.749453, -63.873544 #porto velho
#lat, lon = -8.058493, -34.884819 #recife
#lat, lon = -15.775446, -47.797089 #brasilia
#lat, lon = -15.598669, -56.099130 #cuiabá
#lat, lon = -3.131633, -59.982504 #manaus
#lat, lon = -30.032500, -51.230377 #porto alegre
#lat, lon = -22.911014, -43.209373 #rio de janeiro
#lat, lon = -27.597300, -48.549610 #florianópolis
#lat, lon = -20.812585, -49.380421 #são josé do rio preto
#lat, lon = 2.820848, -60.671958 #boa vista

inclinacao = 10      # degrees
orientacao = 253       # 0 = norte
modelo_transposicao = 'klucher'

losses_parameters = {
    'soiling': 0.0,
    'shading': 0.0,
    'mismatch': 0.0,
    'wiring': 0.0
}

module_parameters = {
    'alpha_sc': 0.00046,
    'beta_oc': -0.0025,
    'gamma_r': -0.0029,
    'a_ref': 1.8,
    'I_L_ref': 14.86,
    'I_o_ref': 2.5e-12,
    'R_s': 0.25,
    'R_sh_ref': 450.0,
    'cells_in_series': 132,
    'STC': potencia_modulo,
    'V_oc_ref': 48.0,
    'I_sc_ref': 14.86,
    'V_mp_ref': 45.0,
    'I_mp_ref': 13.44,
    'A0': -3.47, 'A1': -0.0594, 'A2': 0.0, 'A3': 0.0, 'A4': 0.0,
    'B0': 0.0, 'B1': 0.0, 'B2': 0.0, 'B3': 0.0, 'B4': 0.0, 'B5': 0.0,
    'DTC': 3.0
}

inverter_parameters = {
    'Paco': 75000,   # AC rating
}

# ---------------------------
# Funções utilitárias
# ---------------------------
def ajustar_configuracao_string(num_modulos, preferencia_modules_per_string=8):
    modules_per_string = int(max(1, preferencia_modules_per_string))
    strings = math.ceil(num_modulos / modules_per_string)
    modules_per_string = math.ceil(num_modulos / strings)
    modules_per_string = max(1, modules_per_string)
    strings = max(1, math.ceil(num_modulos / modules_per_string))
    return modules_per_string, strings

def buscar_dados_pvgis(lat, lon, startyear=2018, endyear=2020, timeout=60):
    url = (f"https://re.jrc.ec.europa.eu/api/v5_2/seriescalc?"
           f"lat={lat}&lon={lon}&startyear={startyear}&endyear={endyear}&"
           f"outputformat=json&usehorizon=1&selectrad=1&angle=0&aspect=0")
    try:
        print(f"Buscando PVGIS: {lat},{lon} [{startyear}-{endyear}]")
        r = requests.get(url, timeout=timeout)
        r.raise_for_status()
        data = r.json()
    except Exception as e:
        print("❌ Erro PVGIS:", e)
        return None

    recs = []
    hourly = data.get('outputs', {}).get('hourly', [])
    if not hourly:
        print("⚠️ PVGIS retornou sem 'hourly'.")
        return None

    for rec in hourly:
        try:
            dt = pd.to_datetime(rec['time'], format='%Y%m%d:%H%M', utc=True)
            recs.append({
                'datetime': dt,
                'ghi': float(rec.get('G(i)', 0.0)),
                'dni': float(rec.get('Gb(n)', 0.0)),
                'dhi': float(rec.get('Gd(i)', 0.0)),
                'temp_air': float(rec.get('T2m', 25.0)),
                'wind_speed': float(rec.get('WS10m', 2.0))
            })
        except Exception:
            continue

    df = pd.DataFrame(recs).set_index('datetime')
    if df.empty:
        return None
    df.index = df.index.tz_convert('America/Sao_Paulo')
    print(f"✅ PVGIS OK: {len(df)} registros. GHI média: {df['ghi'].mean():.1f} W/m²")
    return df

def gerar_dados_sinteticos(lat, lon, start='2019-01-01', end='2019-12-31', tz='America/Sao_Paulo'):
    print("Gerando dados sintéticos (clear-sky)...")
    times = pd.date_range(start=start, end=end, freq='1h', tz=tz)
    site = location.Location(lat, lon, tz=tz)
    cs = site.get_clearsky(times)
    df = cs.rename(columns={'ghi': 'ghi', 'dni': 'dni', 'dhi': 'dhi'})
    df['temp_air'] = 25.0
    df['wind_speed'] = 2.0
    return df

# ---------------------------
# Função principal revisada
# ---------------------------
def calcular_sistema_solar(
    lat, lon, inclinacao, azimute, num_modulos,
    losses_parameters, module_parameters, inverter_parameters,
    potencia_modulo=605, modelo_transposicao='klucher',
    modules_per_string_pref=8, startyear=2018, endyear=2020,
    fallback_to_synthetic=True
):
    print("=== Iniciando simulação ===")
    potencia_total_kWp = num_modulos * potencia_modulo / 1000.0
    print(f"Sistema: {num_modulos} x {potencia_modulo} W = {potencia_total_kWp:.2f} kWp")

    modules_per_string = num_modulos
    strings_per_inverter = 1
    total_calculado = modules_per_string * strings_per_inverter
    print(f"Config elétrica sugerida: modules_per_string={modules_per_string}, strings={strings_per_inverter} (total calculado={total_calculado} módulos)")

    Paco = inverter_parameters.get('Paco')
    Pdco_calc = Paco / 0.9811  # eficiência max assumida
    inv_params = inverter_parameters.copy()
    inv_params['Pdco'] = Pdco_calc
    inv_params['pdc0'] = Pdco_calc       # obrigatório para PVWatts
    print(f"Inverter: Paco={Paco} W | Pdco ajustado = {Pdco_calc:.0f} W")

    # PVGIS / fallback
    df = buscar_dados_pvgis(lat, lon, startyear=startyear, endyear=endyear)
    if df is None and fallback_to_synthetic:
        df = gerar_dados_sinteticos(lat, lon, start=f"{startyear}-01-01", end=f"{endyear}-12-31", tz='America/Sao_Paulo')
    if df is None or df.empty:
        print("❌ Não foi possível obter dados meteorológicos. Abortando.")
        return None

    solar_pos = solarposition.get_solarposition(df.index, lat, lon)
    if df['dni'].sum() == 0:
        decomp = irradiance.louche(ghi=df['ghi'], solar_zenith=solar_pos['zenith'], datetime_or_doy=df.index)
        df['dni'] = decomp['dni']
        df['dhi'] = decomp['dhi']

    weather = pd.DataFrame({
        'ghi': df['ghi'],
        'dni': df['dni'],
        'dhi': df['dhi'],
        'temp_air': df['temp_air'],
        'wind_speed': df['wind_speed']
    }, index=df.index)

    system = pvsystem.PVSystem(
        surface_tilt=inclinacao,
        surface_azimuth=azimute,
        module_parameters=module_parameters,
        inverter_parameters=inv_params,
        modules_per_string=modules_per_string,
        strings_per_inverter=strings_per_inverter,
        temperature_model_parameters=TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass'],
        losses_parameters={}
    )

    site = location.Location(latitude=lat, longitude=lon, tz='America/Sao_Paulo')
    mc = modelchain.ModelChain(system, site, transposition_model=modelo_transposicao, aoi_model='no_loss', ac_model='pvwatts')

    print("Executando ModelChain (simulação)...")
    try:
        mc.run_model(weather)
    except Exception as e:
        print("❌ Erro ao rodar ModelChain:", e)
        traceback.print_exc()
        return None

    # Clip AC para inverter
    dc_power = mc.results.dc['p_mp'].fillna(0)
    ac_power = np.minimum(dc_power, Paco)

    # Perdas pós-simulação
    perdas_totais_pct = sum(losses_parameters.values())
    ac_after_losses = ac_power * (1.0 - perdas_totais_pct / 100.0)

    n_anos = df.index.year.nunique()
    annual_energy_kwh = ac_after_losses.sum() / 1000.0 / n_anos
    yield_especifico = annual_energy_kwh / potencia_total_kWp if potencia_total_kWp > 0 else np.nan
    fator_capacidade = (annual_energy_kwh / (potencia_total_kWp * 8760.0)) * 100.0 if potencia_total_kWp > 0 else np.nan

    monthly_kwh = ac_after_losses.resample('M').sum() / 1000.0
    monthly_avg = monthly_kwh.groupby(monthly_kwh.index.month).mean()
    monthly_series = monthly_avg.reindex(range(1,13), fill_value=0.0)

    # Resultados resumidos
    print("\n=== RESULTADO (RESUMO) ===")
    print(f"Geração anual média: {annual_energy_kwh:,.0f} kWh/ano ({n_anos} anos)")
    print(f"Yield específico: {yield_especifico:,.1f} kWh/kWp")
    print(f"Fator de capacidade: {fator_capacidade:.2f} %")
    print(f"Modules/string: {modules_per_string} | strings: {strings_per_inverter} | Pdco usado: {Pdco_calc:.0f} W\n")

    meses = ['Jan','Fev','Mar','Abr','Mai','Jun','Jul','Ago','Set','Out','Nov','Dez']
    print("Geração média mensal (kWh):")
    for i, m in enumerate(meses, 1):
        print(f"  {m}: {monthly_series.get(i, 0.0):,.0f}")

    return {
        'annual_energy_kwh': annual_energy_kwh,
        'yield_especifico': yield_especifico,
        'fator_capacidade': fator_capacidade,
        'monthly_avg_kwh': monthly_series,
        'ac_after_losses': ac_after_losses,
        'weather': weather,
        'modules_per_string': modules_per_string,
        'strings_per_inverter': strings_per_inverter,
        'Pdco_used': Pdco_calc,
        'n_anos': n_anos
    }

# ---------------------------
# Execution wrapper
# ---------------------------
if __name__ == "__main__":
    res = calcular_sistema_solar(
        lat, lon, inclinacao, orientacao, num_modulos,
        losses_parameters, module_parameters, inverter_parameters,
        potencia_modulo=potencia_modulo,
        modelo_transposicao=modelo_transposicao,
        modules_per_string_pref=8,
        startyear=2018,
        endyear=2020,
        fallback_to_synthetic=True
    )

    if res is None:
        print("Simulação não produziu resultado.")
        sys.exit(1)

    try:
        res['monthly_avg_kwh'].to_csv('monthly_avg_kwh.csv', header=['kWh'])
        print("\nArquivo gerado: monthly_avg_kwh.csv")
    except Exception as e:
        print("Aviso: falha ao salvar arquivos:", e)


=== Iniciando simulação ===
Sistema: 22 x 605 W = 13.31 kWp
Config elétrica sugerida: modules_per_string=22, strings=1 (total calculado=22 módulos)
Inverter: Paco=75000 W | Pdco ajustado = 76445 W
Buscando PVGIS: -22.841432,-51.957627 [2018-2020]
✅ PVGIS OK: 26304 registros. GHI média: 221.9 W/m²
Executando ModelChain (simulação)...


/usr/local/lib/python3.12/dist-packages/pvlib/singlediode.py:790: RuntimeWarning: invalid value encountered in multiply
  I[idx_p] = (IL[idx_p] + I0[idx_p] - V[idx_p] * Gsh[idx_p]) / \



=== RESULTADO (RESUMO) ===
Geração anual média: 18,228 kWh/ano (4 anos)
Yield específico: 1,369.5 kWh/kWp
Fator de capacidade: 15.63 %
Modules/string: 22 | strings: 1 | Pdco usado: 76445 W

Geração média mensal (kWh):
  Jan: 2,380
  Fev: 2,121
  Mar: 2,280
  Abr: 1,977
  Mai: 1,615
  Jun: 1,343
  Jul: 1,678
  Ago: 1,764
  Set: 1,966
  Out: 2,203
  Nov: 2,467
  Dez: 1,882

Arquivo gerado: monthly_avg_kwh.csv


/tmp/ipython-input-2996189840.py:208: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_kwh = ac_after_losses.resample('M').sum() / 1000.0
